In [1]:
library("rerddap")
library("ncdf4")

In [2]:
?rerddap

rerddap-package {rerddap},R Documentation


In [3]:
whichSST <- ed_search(query = "ersstv5")

In [4]:
whichSST

# A tibble: 9 × 2
  title                                                               dataset_id
  <chr>                                                               <chr>     
1 NOAA ERSSTv5 (in situ only) (sst.mnmean), 2.0°, 1854-present        noaa_psl_…
2 NOAA ERSSTv5 (in situ only) (sst.mon.ltm), 2.0°, 0001               noaa_psl_…
3 NOAA ERSSTv5 (in situ only) (sst.mon.ltm.1981-2010), 2.0°, 0001     noaa_psl_…
4 NOAA ERSSTv5 (in situ only) (sst.mon.ltm.1991-2020), 2.0°, 0001     noaa_psl_…
5 NOAA ERSSTv5 (in situ only), 2°, Global, Monthly, 1854-present      nceiErsst…
6 NOAA ERSSTv5 (in situ only), 2°, Global, Monthly, 1854-present, Lo… nceiErsst…
7 NOAA Merged Land Ocean Global Surface Temperature Analysis (NOAAGl… noaa_psl_…
8 NOAA Merged Land Ocean Global Surface Temperature Analysis (NOAAGl… noaa_psl_…
9 NOAA Merged Land Ocean Global Surface Temperature Analysis (NOAAGl… noaa_psl_…

In [5]:
info('nceiErsstv5_LonPM180')

<ERDDAP info> nceiErsstv5_LonPM180 
 Base URL: https://upwell.pfeg.noaa.gov/erddap 
 Dataset Type: griddap 
 Dimensions (range):  
     time: (1854-01-01T00:00:00Z, 2023-04-15T00:00:00Z) 
     depth: (0.0, 0.0) 
     latitude: (-88.0, 88.0) 
     longitude: (-180.0, 178.0) 
 Variables:  
     sst: 
         Units: degree_C 
     ssta: 
         Units: degree_C 

In [6]:
sstInfo <- info('nceiErsstv5_LonPM180')
erSST <- griddap(sstInfo, latitude = c(22., 51.), longitude = c(-140., -105), time = c("2017-01-01", "2017-01-02"), fields = 'ssta')

info() output passed to x; setting base url to: https://upwell.pfeg.noaa.gov/erddap



In [7]:
erSST

<ERDDAP griddap> nceiErsstv5_LonPM180
   Path: [/tmp/RtmpLxGcY0/R/rerddap/555cf83de304058d6b67ceda0735f045.nc]
   Last updated: [2023-05-23 10:25:49]
   File size:    [0.01 mb]
   Dimensions (dims/vars):   [4 X 1]
   Dim names: time, depth, latitude, longitude
   Variable names: Extended reconstructed SST anomalies
   data.frame (rows/columns):   [304 X 5]
# A tibble: 304 × 5
   longitude  latitude     depth time                  ssta
   <dbl[1d]> <dbl[1d]> <dbl[1d]> <chr>                <dbl>
 1      -140        22         0 2017-01-15T00:00:00Z 0.498
 2      -138        22         0 2017-01-15T00:00:00Z 0.548
 3      -136        22         0 2017-01-15T00:00:00Z 0.629
 4      -134        22         0 2017-01-15T00:00:00Z 0.698
 5      -132        22         0 2017-01-15T00:00:00Z 0.734
 6      -130        22         0 2017-01-15T00:00:00Z 0.763
 7      -128        22         0 2017-01-15T00:00:00Z 0.821
 8      -126        22         0 2017-01-15T00:00:00Z 0.873
 9      -124        2

In [8]:
erSST$data

longitude,latitude,depth,time,ssta
<dbl[1d]>,<dbl[1d]>,<dbl[1d]>,<chr>,<dbl>
-140,22,0,2017-01-15T00:00:00Z,0.4975815
-138,22,0,2017-01-15T00:00:00Z,0.5484982
-136,22,0,2017-01-15T00:00:00Z,0.6286888
-134,22,0,2017-01-15T00:00:00Z,0.6980591
-132,22,0,2017-01-15T00:00:00Z,0.7338963
-130,22,0,2017-01-15T00:00:00Z,0.7631321
-128,22,0,2017-01-15T00:00:00Z,0.8205128
-126,22,0,2017-01-15T00:00:00Z,0.8727989
-124,22,0,2017-01-15T00:00:00Z,0.8595047


In [9]:
urlBase <- "https://gliders.ioos.us/erddap/"
dataInfo <- rerddap::info("allDatasets", url = urlBase)
parameter <- "allDatasets"

In [10]:
allGlider <- tabledap(dataInfo)

info() output passed to x; setting base url to: https://gliders.ioos.us/erddap



In [11]:
allGlider[0:10,0:3]

,datasetID,accessible,institution
,<chr>,<chr>,<chr>
2,allDatasets,public,IOOS Glider DAC
3,amelia-20180501T0000,public,Virginia Institute of Marine Science - William & Mary
4,amelia-20200825T1929,public,Virginia Institute of Marine Science - William & Mary
5,amelia-20201015T1436,public,Virginia Institute of Marine Science - William & Mary
6,amlr01-20181216T0641-delayed,public,NOAA SWFSC Antarctic Ecosystem Research Division
7,amlr01-20191206T0452-delayed,public,NOAA SWFSC Antarctic Ecosystem Research Division
8,amlr02-20181211T1233-delayed,public,NOAA SWFSC Antarctic Ecosystem Research Division
9,amlr02-20191206T1236-delayed,public,NOAA SWFSC Antarctic Ecosystem Research Division
10,amlr03-20191206T0529-delayed,public,NOAA SWFSC Antarctic Ecosystem Research Division
